<a href="https://colab.research.google.com/github/tmckim/materials-fa23-colab/blob/main/lectures/lec05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Before you start - Save this notebook!

When you open a new Colab notebook from the WebCampus (like you hopefully did for this one), you cannot save changes. So it's  best to store the Colab notebook in your personal drive `"File > Save a copy in drive..."` **before** you do anything else.

The file will open in a new tab in your web browser, and it is automatically named something like: "**Copy of lec05.ipynb**". You can rename this to just the title of the assignment "**lec05.ipynb**". Make sure you do keep an informative name (like the name of the assignment) so that you know which files to submit back to WebCampus for grading! More instructions on this are at the end of the notebook.


**Where does the notebook get saved in Google Drive?**

By default, the notebook will be copied to a folder called “Colab Notebooks” at the root (home directory) of your Google Drive. If you use this for other courses or personal code notebooks, I recommend creating a folder for this course and then moving the assignments AFTER you have completed them.

In [ ]:
# Setup and add files needed to gdrive
# If you restart colab, start by rerunning this cell first!
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

#!mkdir -p '/content/gdrive/My Drive/colab-materials-NS499DataSci-notebooks/'
%cd /content/gdrive/My Drive/colab-materials-NS499DataSci-notebooks/
!rm -r materials-fa23-colab

!git clone https://github.com/tmckim/materials-fa23-colab '/content/gdrive/My Drive/colab-materials-NS499DataSci-notebooks/materials-fa23-colab/'

%cd /content/gdrive/MyDrive/colab-materials-NS499DataSci-notebooks/materials-fa23-colab/lectures/

In [ ]:
# Import packages and other things needed
# Don't change this cell; Just run this cell
# If you restart colab, make sure to run this cell again after the first one above^

from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action='ignore',category=np.VisibleDeprecationWarning)

## Lecture 05 ##

Topics:
- Reviewing functions
- Working with apply and group
- Introduce Python lists
- Show how to build tables from rows instead of columns
- Introduce pivots - a special kind of grouping
- Introduce joins

## Prediction ##

Can we predict how tall a child will grow based on the height of their parents?

To do this we will use the famous [Galton's height dataset](https://galton.org/essays/1880-1889/galton-1886-jaigi-regression-stature.pdf) that was collected to demonstrate the connection between parent's heights and the height of their children.


In [ ]:
# Read in the table of data
families = Table.read_table('family_heights.csv')
families

## Exploring the Data ##

**Exercise**: Add a column `parent average` containing the average height of both parents.

In [ ]:
families = families.with_columns(
    'parent average', (families.column('father') + families.column('mother'))/2)
families

What is the relationship between a child's height and the average parent's height?

**Exercise**: Make a scatterplot showing the relationship between `parent average` and the `child` height

In [ ]:
# Scatterplot
families.scatter('parent average', 'child')

Questions:
1. Do we observe a relationship between child and parent height?
2. Would a line plot help reveal that relationship?

In [ ]:
# How do we draw a line plot?
families.plot('parent average', 'child', marker = 'o')

## Making a Prediction ##

If we wanted to predict the height of a child given the height of the parents, we could look at the height of children with parents who have a similar average height.

In [ ]:
# Example parent height values
parent1_height = 5*12 + 10 # 5 ft 10 inches
parent2_height = 5*12+7 # 5 ft 7 inches

In [ ]:
# Calc parent average
calc_parent_avg = (parent1_height + parent2_height)/2
calc_parent_avg

Let's look at parents that are within 1 inch of our height

In [ ]:
window = 1
lower_bound = calc_parent_avg - window
upper_bound = calc_parent_avg + window

In [ ]:
# Scatterplot
families.scatter('parent average', 'child')
# you don't need to know the details of this plotting code yet
plots.plot([lower_bound, lower_bound], [50, 85], color='red', lw=2)
plots.plot([calc_parent_avg, calc_parent_avg], [50, 85], color='orange', lw=2);
plots.plot([upper_bound, upper_bound], [50, 85], color='red', lw=2);

**Exercise**: Create a function that takes an average of the parent heights and returns ***an array of all the children's heights*** that are within the window of the parent's average height.

In [ ]:
def similar_child_heights(parent_average):
    lower_bound = parent_average - window
    upper_bound = parent_average + window
    return (
        families
        .where("parent average", are.between(lower_bound, upper_bound))
        .column("child")
    )

Test the function:

In [ ]:
# window = 1.0 originally
# window = 5.0 # smaller or larger- what happens?
similar_child_heights(calc_parent_avg)

**Exercise**: Create a function to predict the child's height as the average of the height of children within the window of the average parent height.

In [ ]:
# Create a new function that uses our previous function
def predict_child_height(parent_average):
    return np.average(similar_child_heights(parent_average))

In [ ]:
# Test the function to get our mean
predict_child_height(calc_parent_avg)

Let's plot the predicted height as well as the distribution of children's heights:

In [ ]:
# window = 1.0
similar = similar_child_heights(calc_parent_avg) # array of heights from our first function
predicted_height = predict_child_height(calc_parent_avg) # single # for the average from our second function

print("Mean:", predicted_height)
Table().with_column("child", similar).hist("child", bins= 20) # new table with the values in the array to plot with hist
plots.plot([predicted_height, predicted_height], [0,0.1], color = "red") # added a line for the average that we calculated

## Prediction Accuracy: Evaluating the Predictions ##

To evalute the predictions, let's see how the predictions compare to the actual heights of all the children in our dataset.

**Exercise**: Apply the function (using `appy`) to all the parent averages in the table and save the result to the `predicted` column.

In [ ]:
# window = 1.0
families = families.with_column(
    "predicted", families.apply(predict_child_height, "parent average")
)
families

**Exercise**: Construct a scatterplot with the `parent average` height on the x-axis and the `child` height and the `predicted` height on the y-axis.

In [ ]:
families.select("parent average", "child", "predicted").scatter("parent average")

**Discussion**: What do we see in this plot?

**Exercise**: Define a function to compute the error (the difference) between the predicted value and the true value. Apply that function to the table adding a column containing the `error`. Then construct a histogram of the errors.

In [ ]:
def error(predicted, true_value):
    return predicted - true_value

families = families.with_column(
    "error", families.apply(error, "predicted", "child"))
families

Visualize the distribution of errors:

In [ ]:
families.hist('error')

**Exercise**: Overlay the histograms of the error for male and female children.

In [ ]:
families.hist('error', group='sex')

**Discussion**: What do we observe?

# Discussion Question: Building a Better Predictor

**Exercise**: Implement a new height prediction function that considers the averages of the height of children with the same sex and whose parents had a similar height.<br><br>

Start by reviewing our previous function:
```
def similar_child_heights(parent_average):
    lower_bound = parent_average - window
    upper_bound = parent_average + window
    return np.average(
        families
        .where("parent average", are.between(lower_bound, upper_bound))
        .column("child")
    )
```



In [ ]:
def predict_child_height_with_sex(parent_average):

In [ ]:
# Test out our function

In [ ]:
# Test out our function

**Exercise**: Apply the better predictor to the table and save the predictions in a column called `predicted with sex`.

In [ ]:
families= families.with_column(
    "predicted with sex", families.apply(predict_child_height_with_sex, "parent average", "sex")
)
families

**Exercise**: Construct a histogram of the new errors broke down by the sex of the child.

In [ ]:
families = families.with_column(
    "error with sex",  families.apply(error, "predicted with sex", "child")
)
families.hist("error with sex", group="sex")

The previous plot for comparison:

In [ ]:
families.hist("error", group="sex")

## Grouping by One Column ##

In [ ]:
# Load in our ice cream data
cones = Table.read_table('cones_rating.csv')
cones

**Exercise**: Use the `group` function to determine the number of cones with each flavor.

In [ ]:
cones.group('Flavor')

**Exercise**: Use the `group` function to compute the average price of cones for each flavor.

In [ ]:
cones.group('Flavor', np.average)

In [ ]:
cones.select('Flavor', 'Price').group('Flavor', np.mean)

**Exercise**: Use the `group` function to compute the minimum price of cones for each flavor.

In [ ]:
cones.group('Flavor', np.min)

In [ ]:
cones.group('Flavor', min)

In [ ]:
cones

## Lists

In [ ]:
# A simple list
[1, 5, 'hello', 5.0]

In [ ]:
# Another list
another_list = [1, 5, 'hello', 5.0, make_array(1,2,3)]
another_list

You have to use square brackets when accessing elements from a list (you can't use `item`)


In [ ]:
another_list[2]

In [ ]:
another_list[4][1]

Recall that for arrays you could do either. (In the real world, most people would probably use square brackets).

In [ ]:
my_array = make_array('Apple', 'Orange', 'Banana')
my_array[1]

Same thing as:

In [ ]:
my_array.item(1)

## Rows from Lists ##

We need lists to be able to add a row to our table. Why?


Here we make a table with a column (we've been doing this alot already).

In [ ]:
Table().with_columns('Numbers', [1, 2, 3])

Here we make an empty table with just column labels. (This is new!)

In [ ]:
# Empty table with columns and no rows
drinks = Table(['Drink', 'Cafe', 'Price'])
drinks

Here we add rows to our table. Each row is a `List`of elements -- one for each column.



In [ ]:
drinks = drinks.with_rows([
    ['Milk Tea', 'RareTea', 5.5],
    ['Espresso', 'Buzzed Coffee',  1.75],
    ['Latte',    'Buzzed Coffee',  3.25],
    ['Espresso', "Starbucks",   2]
])
drinks

## Grouping by Two Columns: Cross Classification ##

Classify the cones by both `flavor` and `color`

In [ ]:
# Default without specifying an argument is to add a column of the count
cones.group(['Flavor', 'Color'])

Specify how to `collect` the items with `sum`

In [ ]:
cones.group(['Flavor', 'Color'], sum)

In [ ]:
# Average
cones.group(['Flavor', 'Color'], np.mean)

## Pivot Tables ##

When grouping by two columns it will sometimes be easier to re-arrange the table so that the columns correspond to the values of one of the original grouped columns.

Here is an example:

In [ ]:
# Pivot using the default collect = count
cones.pivot('Color', 'Flavor')

Explicitly naming the function arguments. You wouldn't do this in practice, but it helps to highlight the meaning of the four things we passed to the function above.

In [ ]:
cones.pivot(
    columns = 'Color', # column whose unique values become columns
    rows = 'Flavor', # column whose unique values become rows
    values = 'Price', # column that will be used to do computing
    collect = np.average) # how to aggregate the values

## Joins ##

We use join to **join** two tables based on a similar column.

In [ ]:
drinks

In [ ]:
discounts = Table().with_columns(
    'Coupon % off', make_array(10, 25, 5),
    'Location', make_array('RareTea', 'Buzzed Coffee', 'RareTea')
)
discounts

**Exercise**: Combine the `drinks` and `discounts` tables into a new table which has all possible discounts for each drink.

In [ ]:
combined = drinks.join('Cafe', discounts, 'Location')
combined

**Exercise**: Calculate the lowest rate you can get for each drink using the available coupons.

In [ ]:
discounted_frac = 1 - combined.column('Coupon % off') / 100
(
combined.with_column('Discounted Price', (combined.column('Price') * discounted_frac))
.select('Drink', 'Discounted Price')
.group('Drink', min)
)

**Exercise**: What happens if I run the following? How many rows will it produce?

In [ ]:
drinks.join('Cafe', drinks, 'Cafe')